# Lineage Demo

Source Dataset: https://www.kaggle.com/datasets/pschale/mlb-pitch-data-20152018?select=2019_atbats.csv



In [0]:
%python
target_catalog = "lgeorge"
target_schema = "lineage_demo"

In [0]:
%py
spark.sql("CREATE CATALOG IF NOT EXISTS {target_catalog}")
spark.sql("CREATE SCHEMA IF NOT EXISTS {target_schema}")

In [0]:
%python
spark.sql(f"USE CATALOG {target_catalog}")
spark.sql(f"USE SCHEMA {target_schema}")


In [0]:
CREATE VOLUME IF NOT EXISTS raw_data

In [0]:
%python
from zipfile import ZipFile 
  
# loading the temp.zip and creating a zip object 
with ZipFile("/Volumes/lgeorge/lineage_demo/raw_data/data_files.zip", 'r') as zipped_file: 
  
    # Extracting all the members of the zip  
    # into a specific location. 
    zipped_file.extractall( 
        path="/Volumes/lgeorge/lineage_demo/raw_data/") 

In [0]:
%py
import glob

for filepath in glob.glob('/Volumes/lgeorge/lineage_demo/raw_data/*.csv'):
    if '2019' not in filepath:
      
      table_name = filepath.split("/")[-1][:-4]

      df = spark.read.format("csv").option("header", True).option("inferSchema", True).load(filepath)
      df.write.format("delta").mode("overwrite").saveAsTable(table_name)


In [0]:
CREATE TABLE atbat_general
AS
SELECT
  ab.g_id as game_id,
  ab.ab_id,
  concat_ws(" ", b.first_name, b.last_name) as batter_name,
  concat_ws(" ", p.first_name, p.last_name) as pitcher_name,  
  g.umpire_HP,
  ab.event as atbat_event,
  ab.inning,
  ab.o as out,
  ab.top as top_of_inning,
  g.home_team,
  g.away_team,
  g.weather,
  g.wind
FROM
  atbats ab
LEFT JOIN player_names b ON ab.batter_id = b.id
LEFT JOIN player_names p ON ab.pitcher_id = p.id
LEFT JOIN games g ON ab.g_id = g.g_id

In [0]:
CREATE TABLE ejected_players_summary
AS
SELECT
  e.player_id as player_id,
  concat_ws(" ", p.first_name, p.last_name) as ejected_player_name,
  count(*) as ejected_count
FROM
  player_names p
INNER JOIN ejections e ON p.id = e.player_id
GROUP BY player_id, ejected_player_name
ORDER BY ejected_count DESC;

In [0]:
SELECT * FROM ejections;

In [0]:
SELECT * FROM lgeorge.lineage_demo.pitches